# Example notebook to calculate new Fisher matrices

This notebook provides an example of how to calculate new Fisher matrices, including the numerical derivatives of the CMB and BAO theory with respect to cosmological parameters. It does this using the `Fisher` class of `hdfisher/fisher.py`.

Below, we import the required code to run this notebook:

In [1]:
import os
from IPython.display import display
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from hdfisher import fisher
plt.rcParams['figure.dpi'] = 250

# Code initialization and setup

There is one __required__ argument when initializing the `Fisher` class:
- `fisher_dir`: The _absolute_ path to the directory that will hold the output. If the directory does not exist, it will be created upon initialization, along with the following files and directories:
  1. `fiducial_params.yaml` and `step_sizes.yaml` : Files holding the fiducial parameter values, and the step sizes used for each varied parameter, respectively. 
      - This information is used to calculate the numerical derivatives.
      - We have provided files containing our fiducial parameter values (including for the CAMB accuracy parameters) and Fisher step sizes, which are used by default: see `dataconfig.Data.fiducial_param_file` and `dataconfig.Data.fiducial_fisher_steps_file`.
      - See `param_file` and `fisher_steps_file` below if you would like to use your own. You can look at the files that will be saved in the `fisher_dir` for an example.
  2. `theory`: An empty directory that will be used to store the theory calculation when each parameter is varied.
  3. `derivs`: An empty directory that will be used to store the numerical derivatives of the theory with respect to each parameter.
  4. `fisher_matrices`: An empty directory that can be used to store any Fisher matrices that are computed from the derivatives in the `derivs` directory.
  
Below, we set the `fisher_dir` to be a directory (relative to this notebook) named `example_output/hd_example`.

The following arguments are _optional_; we use the default values in this example:

- `overwrite` (default=`False`): Pass `overwrite=True` to overwrite existing files in the `fisher_dir` and its sub-directories. Otherwise, try to load results that have already been computed.
- `param_file` (default=`None`): The name (including its absolute path) of a YAML file containing parameter names and their fiducial values, or leave `param_file=None` to use the default described above. 
  - __Note__ that the CAMB accuracy parameters are also set in this file.
  - The parameter names must be names that can be passed to the CAMB function `camb.set_params`; however, we also use an alternative name or definition for a few parameters:
    1. `theta`, which is $100 \theta_\mathrm{MC}$; the corresponding parameter in CAMB is `100 * cosmomc_theta`.
    2. `logA`, which is $\ln(10^{10} A_\mathrm{s})$
    3. `logTagn`, which is the HMCode2020 baryonic feedback parameter $\log_{10}(T_\mathrm{AGN}/\mathrm{K})$, named `HMCode_logT_AGN` in CAMB (this is mostly for backwards compatibility).
- `fisher_steps_file` (default=`None`): The name (including its absolute path) of a YAML file containing parameter names and information about the step size to use when varying each, or leave `fisher_steps_file=None` to use the default described above. 
  - Each parameter in the `fisher_steps_file` must have a fiducial value given in the `param_file`, but you do not need to specify a step size for all parameters appear in the `param_file`. 
  - For example, to vary $n_\mathrm{s}$ by $\pm$ 1% of its fiducial value of 0.9649, the `fisher_steps_file` should contain one of the following blocks:
```
ns:
    step_size: 0.01
    step_type: relative
```
or
```
ns:
    step_size: 0.009649
    step_type: absolute
```
- `fisher_params` (default=`None`): A list of parameter names to vary when calculating the derivatives of the theory. This may be a subset of the parameters in the `fisher_steps_file`, or left as `None` to vary all of them.
- `feedback` (default=`False`): Only used to find the defaults if the `param_file` and/or `fisher_steps_file` aren't given. If `True`, use the HMCode2020 baryonic feedback model instead of the HMCode2016 CDM-only model.
- `use_H0` (default = `False`): This is used when the `fisher_steps_file` described above contains _both_ `H0` and `cosmomc_theta` (or `theta`, defined as `100 * cosmomc_theta`): only one of these can be passed to CAMB, and therefore only one can remain fixed while the other parameters are varied.  If `use_H0 = False`, $\theta_\mathrm{MC}$ is used, and if `use_H0 = True`,  $H_0$ is used.

We also provide the following extra optional arguments:

- `exp` (default=`'hd'`): Used to load the covariance matrix for the mock data spectra, and determine the multipole ranges used in the calculation. Must be `'hd'`, `'s4'`, or `'so'` for CMB-HD-like, CMB-S4-like, or SO-like configurations, respectively. 
- `hd_data_version` (default=`'latest'`): The version number used to load the CMB-HD mock data from the `hdMockData` [repository](https://github.com/CMB-HD/hdMockData); please see that repository for more information about the data versions. By default, the latest CMB-HD data is used.
    - If you are running new forecasts, please use the latest CMB-HD data.
    - We provide this option to allow you to reproduce the results of MacInnis et. al. (2023), which used version `'v1.0'`.
- `hd_lmax` (default=`None`): Only for version `'v1.0'` of the mock CMB-HD delensed data. This can be an integer value lower than the baseline of $\ell_{max}, L_{max} = 20,100$, which is used by default if `hd_lmax=None`. The options are `1000`, `3000`, `5000`, and `10000`.
- `include_fg` (default=`True`): Only for version `'v1.0'` of the mock CMB-HD lensed data (i.e., no delensing). Set `include_fg=False` to calculate a Fisher matrix without including the effects of residual extragalactic foregrounds in temperature.

In [2]:
# set the `fisher_dir` that holds the output:
fisher_root = 'hd_example'
output_dir = os.path.join(os.getcwd(), 'example_output')
fisher_dir = os.path.join(output_dir, fisher_root)

# initialize the `Fisher` class:
fisherlib = fisher.Fisher(fisher_dir)

# Calculating new Fisher derivatives

Before calculating new Fisher matrices, you must calculate new Fisher derivatives first. This is done using the `calculate_fisher_derivs` method of the `Fisher` class initialized above. 
 

__Note__ that there are two ways to do the calculation:

1. With MPI: You can calculate the derivatives in parallel using the Python script `example_calculate_fisher_derivs.py`. This will only take a few minutes. For example, to use 4 parallel MPI processes, run the following (or the equivalent command for your machine), and then proceed to the next section:

```
mpirun -np 4 python example_calculate_fisher_derivs.py
```

On NERSC, you will need to request an interactive node, and run, for example,

```
export OMP_NUM_THREADS=64
srun -n 4 -c 64 python example_calculate_fisher_derivs.py
```

Please refer to the NERSC documentation for latest commands.

2. Without MPI: You can un-comment and run the code in the following cell. Note that this will take about 30 minutes, which is significantly longer than the first option.

In [3]:
#fisherlib.calculate_fisher_derivs()

# Calculating new Fisher matrices

Once you've calculated the derivatives of the CMB and BAO theory with respect to parameters, you can calculate Fisher matrices for any set of those parameters. This is done using the `get_fisher` method of the `Fisher` class (which can now be re-initialized from just the single required argument described above). 

The `get_fisher` function returns a tuple with two items: the first is the Fisher matrix (as a `numpy` array), and the second is a list of parameter names corresponding to the rows and columns of the Fisher matrix.

The (optional) input argumemts for `Fisher.get_fisher` are:
- `cmb_type` (default = `delensed`): Set `cmb_type='delensed'` to forecast using delensed CMB power spectra. For CMB-HD, we also provide the option to set `cmb_type='lensed'` to forecast using lensed CMB power spectra for the baseline multipole range $\ell,~L \in [30,~20000]$. 
- `params` (default = `None`): This is an optional list of names of parameter to include in the Fisher matrix. By default, all parameters that you have calculated derivatives for will be used; however, see the note below about $H_0$ and $\theta_\mathrm{MC}$.
- `priors` (default = `None`): A dictionary of the form `{param: prior_width}`, where `param` is the parameter name and `prior_width` is the width of the Gaussian prior to apply.
- `use_H0` (default = `None`): Used to override the value of `use_H0` passed when initializing the `Fisher` class; see the note below.
- `with_desi` (default = `False`): Whether to include mock DESI BAO, by adding its Fisher matrix to the CMB Fisher matrix being calculated.
- `save` (default = `False`) and `fname` (default = `None`): Set `save=True` to save the Fisher matrix to the `fisher_matrices` subdirectory of your `fisher_dir` directory. Otherwise, the Fisher matrix won't be saved. If `save=True`, you must also pass a file name (without a full path) to save the matrix to.


__Note__: If you have calculated the derivatives twice and saved them in the same directory, once with `use_H0=False` and once with `use_H0=True`, you can still re-initialize the `Fisher` class with its single required argument (which, by default, will set `use_H0=False`). If you would like to include $H_0$ in your Fisher matrix, you must override the default by passing `use_H0=True` to `get_fisher`.


If you're only interested in the parameter uncertainties, you can instead use the `get_fisher_errors` method, which takes the same arguments as `get_fisher`.

In [4]:
use_H0 = False
cmb_type = 'delensed'
errs = fisherlib.get_fisher_errors(cmb_type,  
                                   priors={'tau': 0.007}, 
                                   use_H0=use_H0)
errs_with_desi = fisherlib.get_fisher_errors(cmb_type,  
                                   priors={'tau': 0.007}, 
                                   use_H0=use_H0,
                                   with_desi=True)

# load pre-computed results for comparison:
hd_fmat, params = fisherlib.load_example_hd_fisher(cmb_type=cmb_type, use_H0=use_H0, with_desi=False)
hd_errs = fisher.get_fisher_errors(hd_fmat, params)
    
hd_desi_fmat, params = fisherlib.load_example_hd_fisher(cmb_type=cmb_type, use_H0=use_H0, with_desi=True)
hd_desi_errs = fisher.get_fisher_errors(hd_desi_fmat, params)




Define some functions to print out the results:

In [5]:
# order of parameters in tables
param_list = ['ombh2', 'omch2', 'logA', 'ns', 'tau', 'H0', 'nnu', 'mnu']

def ratio(dict1, dict2, fill_value=None):
    """Returns a dictionary with the ratio of the values in `dict1` to those 
    in `dict2` for each key they share in common. If the keys don't match, you
    can pass a `fill_value` as a placeholder for any keys not in both dictionaries;
    otherwise leave it as `None` to exclude those keys from the returned dictionary."""
    rdict = {}
    keys1 = list(dict1.keys())
    keys2 = list(dict2.keys())
    all_keys = list(set(keys1 + keys2))
    for key in all_keys:
        if (key in dict1) and (key in dict2):
            rdict[key] = dict1[key] / dict2[key]
        elif fill_value is not None:
            rdict[key] = fill_value
    return rdict


def print_table(list_of_dicts, list_of_labels, title=None, list_of_keys=None):
    """
    Display a table of keys and values held in each dictionary in the `list_of_dicts`.
    
    Parameters
    ----------
    list_of_dicts : list of dict
        A list of dictionaries.
    list_of_labels : list of str
        A list holding a label for each dictionary in `list_of_dicts`, in
        the same order that they appear in `list_of_dicts`.
    title : str or None, default=None
        A title that is printed out above the table.
    list_of_keys : list or None, default=None
        A list of keys to use in the table. The table will only contain 
        values corresponding to the keys in `list_of_keys`, and in the same
        order. If `None`, all keys are used. In either case, the dictionaries
        in `list_of_dicts` do not need to contain the same set of keys.
    
    """
    table = pd.DataFrame(list_of_dicts, index=list_of_labels, columns=list_of_keys)
    table = table.T
    if title is not None:
        print('\n', title) 
    display(table)

Print out a table comparing the new results to the pre-computed baseline case:

In [6]:
print('\nComparison of new Fisher forecasts with the precomputed baseline case:')


table_list = [hd_errs, errs, ratio(errs, hd_errs)]
label_list = [f'Precomputed HD {cmb_type}', 
              f'New HD {cmb_type} forecast', 'New / Precomputed ratio']
print_table(table_list, label_list)

table_list = [hd_desi_errs, errs_with_desi, ratio(errs_with_desi, hd_desi_errs)]
label_list = [f'Precomputed HD {cmb_type} + DESI', 
              f'New HD {cmb_type} + DESI forecast', 'New / Precomputed ratio']
print_table(table_list, label_list)


Comparison of new Fisher forecasts with the precomputed baseline case:


,Precomputed HD delensed,New HD delensed forecast,New / Precomputed ratio
logA,0.010339,0.010339,1.0
mnu,0.042297,0.042297,1.0
nnu,0.014701,0.014701,1.0
ns,0.002095,0.002095,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000549,0.000549,1.0
tau,0.005344,0.005344,1.0
theta,0.000074,0.000074,1.0


,Precomputed HD delensed + DESI,New HD delensed + DESI forecast,New / Precomputed ratio
logA,0.009724,0.009724,1.0
mnu,0.024218,0.024218,1.0
nnu,0.013776,0.013776,1.0
ns,0.001451,0.001451,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000407,0.000407,1.0
tau,0.005201,0.005201,1.0
theta,0.000059,0.000059,1.0
